## Procesamiento de Lenguage Natural
####Taller #10: Twitter

In [1]:
import pandas as pd 
import tweepy

In [23]:
## Leer llaves
keys = pd.read_csv("keys.csv", header=None)
keys = dict(zip(keys[0],keys[1]))


In [24]:
# Autorizar a través API
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [28]:
#Recolectar datos
tweets = tweepy.Cursor(api.search,
                       q=['Colombia Uruguay'],
                       lang='es',
                       geocode=f'9.304744,-75.397778,45mi',
                       tweet_mode='extended',
                       include_rts=False).items(100)

In [29]:
t = []

for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Bogota")
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t.append(temp)

In [30]:
data = pd.DataFrame(t)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweet,date,city,user_location,id,link
0,Viernes 13 para olvidar: Uruguay goleó a Colom...,2020-11-14 03:45:32,Bogota,"Montería, Colombia",1327457613232054272,https://twitter.com/i/web/status/1327457613232...
1,@ESPNArgentina Colombia revive muertos la elim...,2020-11-14 02:55:13,Bogota,"Montería, Colombia",1327444949231685632,https://twitter.com/i/web/status/1327444949231...
2,Y estaba más cerca Uruguay del 4 que Colombia ...,2020-11-14 02:47:59,Bogota,Sincelejo,1327443129763553280,https://twitter.com/i/web/status/1327443129763...
3,"Tras 7 meses sin fútbol, viene de marcar su PR...",2020-11-14 02:37:57,Bogota,"Córdoba, Colombia",1327440604633108484,https://twitter.com/i/web/status/1327440604633...
4,#balondividido ¿Por qué Perdió Colombia? Te me...,2020-11-14 01:49:06,Bogota,Sincelejo,1327428309521756162,https://twitter.com/i/web/status/1327428309521...


In [31]:
data = data[['tweet', 'date', 'city']]
data.head()

,tweet,date,city
0,Viernes 13 para olvidar: Uruguay goleó a Colom...,2020-11-14 03:45:32,Bogota
1,@ESPNArgentina Colombia revive muertos la elim...,2020-11-14 02:55:13,Bogota
2,Y estaba más cerca Uruguay del 4 que Colombia ...,2020-11-14 02:47:59,Bogota
3,"Tras 7 meses sin fútbol, viene de marcar su PR...",2020-11-14 02:37:57,Bogota
4,#balondividido ¿Por qué Perdió Colombia? Te me...,2020-11-14 01:49:06,Bogota


In [32]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [33]:
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

data.head()

,tweet,date,city,clean,links,hashtags,mentions
0,Viernes 13 para olvidar: Uruguay goleó a Colom...,2020-11-14 03:45:32,Bogota,Viernes 13 para olvidar: Uruguay goleó a Colom...,"[https://t.co/c0voMwLeC2, https://t.co/jhsnHMz...",[],[]
1,@ESPNArgentina Colombia revive muertos la elim...,2020-11-14 02:55:13,Bogota,Colombia revive muertos la eliminatoria pasad...,[],[],[@ESPNArgentina]
2,Y estaba más cerca Uruguay del 4 que Colombia ...,2020-11-14 02:47:59,Bogota,Y estaba más cerca Uruguay del 4 que Colombia ...,[],[],[]
3,"Tras 7 meses sin fútbol, viene de marcar su PR...",2020-11-14 02:37:57,Bogota,"Tras 7 meses sin fútbol, viene de marcar su PR...",[https://t.co/JFaP3vjfwT],"[#ManUnited., #Qatar2022.]",[]
4,#balondividido ¿Por qué Perdió Colombia? Te me...,2020-11-14 01:49:06,Bogota,¿Por qué Perdió Colombia? Te meten un gol en ...,[],[#balondividido],[]


In [37]:
import emoji
import regex
def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

data['emojis'] = data['clean'].apply(lambda text: get_emojis(text))

In [38]:
data.head()

,tweet,date,city,clean,links,hashtags,mentions,emojis
0,Viernes 13 para olvidar: Uruguay goleó a Colom...,2020-11-14 03:45:32,Bogota,Viernes 13 para olvidar: Uruguay goleó a Colom...,"[https://t.co/c0voMwLeC2, https://t.co/jhsnHMz...",[],[],[]
1,@ESPNArgentina Colombia revive muertos la elim...,2020-11-14 02:55:13,Bogota,Colombia revive muertos la eliminatoria pasad...,[],[],[@ESPNArgentina],[]
2,Y estaba más cerca Uruguay del 4 que Colombia ...,2020-11-14 02:47:59,Bogota,Y estaba más cerca Uruguay del 4 que Colombia ...,[],[],[],[]
3,"Tras 7 meses sin fútbol, viene de marcar su PR...",2020-11-14 02:37:57,Bogota,"Tras 7 meses sin fútbol, viene de marcar su PR...",[https://t.co/JFaP3vjfwT],"[#ManUnited., #Qatar2022.]",[],"[📍, 🇨🇴]"
4,#balondividido ¿Por qué Perdió Colombia? Te me...,2020-11-14 01:49:06,Bogota,¿Por qué Perdió Colombia? Te meten un gol en ...,[],[#balondividido],[],[]


Visualizaciones usando Plotly

In [43]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords_sp = stopwords.words('spanish')

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    return texto

tfidf_vect = TfidfVectorizer(preprocessor=pre_procesado, ngram_range=(1,1), max_features=100)
tfidf = tfidf_vect.fit_transform(data.clean.values)

In [44]:
from sklearn.metrics.pairwise import cosine_similarity

voc = [k for k,v in sorted(tfidf_vect.vocabulary_.items(), key=lambda kv: kv[1])]
temp = pd.DataFrame(tfidf.toarray())
temp = cosine_similarity(temp.T.values)
temp = pd.DataFrame(temp)
temp.columns = voc
temp.index = voc
temp

,agua,aquí,argentina,así,atención,barranquilla,bien,chile,colombia,conozca,...,través,tricolor,técnico,uruguay,vamos,ver,viene,viernes,vs,vía
agua,1.0,0.000000,0.000000,0.000000,0.000000,0.273201,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
aquí,0.0,1.000000,0.000000,0.141378,0.825120,0.087667,0.0,0.000000,0.094867,0.660009,...,0.0,0.0,0.216599,0.097939,0.0,0.000000,0.000000,0.000000,0.119088,0.0
argentina,0.0,0.000000,1.000000,0.000000,0.000000,0.073427,0.0,0.269154,0.171767,0.000000,...,0.0,0.0,0.000000,0.082612,0.0,0.260252,0.000000,0.000000,0.000000,0.0
así,0.0,0.141378,0.000000,1.000000,0.171342,0.000000,0.0,0.000000,0.099496,0.491297,...,0.0,0.0,0.000000,0.102717,0.0,0.000000,0.000000,0.430759,0.000000,0.0
atención,0.0,0.825120,0.000000,0.171342,1.000000,0.106248,0.0,0.000000,0.078277,0.799895,...,0.0,0.0,0.262506,0.080811,0.0,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ver,0.0,0.000000,0.260252,0.000000,0.000000,0.000000,0.0,0.000000,0.092918,0.000000,...,0.0,0.0,0.000000,0.095927,0.0,1.000000,0.000000,0.250695,0.092709,0.0
viene,0.0,0.000000,0.000000,0.000000,0.000000,0.032874,0.0,0.000000,0.058939,0.000000,...,0.0,0.0,0.000000,0.060847,0.0,0.000000,1.000000,0.000000,0.196073,0.0
viernes,0.0,0.000000,0.000000,0.430759,0.000000,0.149709,0.0,0.000000,0.107527,0.000000,...,0.0,0.0,0.000000,0.111009,0.0,0.250695,0.000000,1.000000,0.046562,0.0
vs,0.0,0.119088,0.000000,0.000000,0.000000,0.104933,0.0,0.000000,0.146363,0.000000,...,0.0,0.0,0.000000,0.126069,0.0,0.092709,0.196073,0.046562,1.000000,0.0


In [45]:
trace = go.Heatmap(z=temp.values.tolist(),
                   x=temp.index.values,
                   y=temp.columns.values,
                   colorscale='Oranges')

layout = go.Layout(title='🔥 Mapa de calor entre palabras 🔥',
                   width=800, height=800)

fig = go.Figure(data=[trace],layout=layout)
iplot(fig)